In [29]:
!pip install -r requirements.txt #--upgrade
apikey = "7RA6dB7O1NBSWBjmFGuP4DlIZP8DeWqeHW3Rrkr8"
import fooddatacentral as fdc
import pandas as pd
import requests
import time

df = pd.read_csv("trader_joes_enriched.csv") # change this if u want to !!


for idx, row in df.iterrows():
    fdc_id = row["fdc_id"]  

data_list = []

for fdcid in df["fdc_id"]:
    try:
        nutrient = fdc.nutrients(apikey, fdc_id=int(fdcid))

        cols = [f"{idx} ({row['Units']})" for idx, row in nutrient.iterrows()]
        data = dict(zip(cols, nutrient["Quantity"].values))
        
        data["fdc_id"] = int(fdcid)
        data_list.append(data)
    except Exception as e:
        # print(f"Failed on FDC ID {fdcid}, error: {e}")

        continue

df2 = pd.DataFrame(data_list)
df2.to_csv("samsung_galaxy_ultra_nutritional_data_3.csv", index=False)


DONE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog


nutrition = pd.read_csv("samsung_galaxy_ultra_nutritional_data_3.csv")
minimum_req = pd.read_csv("minimum_req.csv")
ingredients = pd.read_csv("hflc_tj.csv") # change this for different shit
shitty = pd.read_csv("USDA-305tj(Sheet1).csv")


nutrition_with_prices = nutrition.merge(ingredients[['GTIN/UPC', 'fdc_id', 'Price']], on='fdc_id', how='left')


nutrition_with_prices = nutrition_with_prices.merge(
    shitty[['GTIN/UPC', 'Name']], 
    on='GTIN/UPC',
    how='left'
)

nutrition_with_prices.dropna(subset=['Price'], inplace=True)

nutrients_to_optimize = minimum_req['Nutrition'].values

available_nutrients = [col for col in nutrition_with_prices.columns if col.split(" (")[0] in nutrients_to_optimize]


costs = nutrition_with_prices['Price'].values

A = nutrition_with_prices[available_nutrients].fillna(0).values.T  

b = minimum_req.set_index('Nutrition').loc[
    [col.split(" (")[0] for col in available_nutrients], "F 19-30"
].apply(pd.to_numeric, errors='coerce').fillna(0).values.flatten()


if A.shape[0] != b.shape[0]:
    if A.shape[0] > b.shape[0]:
        b = np.pad(b, (0, A.shape[0] - b.shape[0]), mode='constant', constant_values=0)
    else:
        b = b[:A.shape[0]]

bounds = [(0, None)] * len(costs)

result = linprog(c=costs, A_ub=-A, b_ub=-b, bounds=bounds, method='highs')

if result.success:
    nutrition_with_prices['Optimal Quantity'] = result.x
    selected_foods = nutrition_with_prices[nutrition_with_prices['Optimal Quantity'] > 0]
    
    solution_df = selected_foods[['Name', 'Price', 'Optimal Quantity']]

    total_cost = result.fun  # Minimum cost found
else:
    solution_df = pd.DataFrame({"Error": ["Optimization Failed"]})
    total_cost = None

print(solution_df)

if total_cost is not None:
    print(f"Minimum Total Cost: ${total_cost:.2f}")

                 Error
0  Optimization Failed
